In [1]:
import json
import visualize
import os
import context as c, score
import pandas as pd

tokenizer = score.get_tokenizer()
dro_tokenizer = score.get_tokenizer("test-mlm/")

/home/gussuvmi@GU.GU.SE/venv/lib64/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load(file_name):
    with open(f"Results/raw/{file_name}", "r") as f:
        res = json.load(f)
    return res

## SELF-DEBIAS

In [3]:
s_debiased = load("sdb_short.json")
l_debiased = load("sdb_long.json")
s_antonym = load("ant_short_probs.json")
l_antonym = load("ant_long_probs.json")

s_raw = visualize.get_sdb_df(s_debiased, c.context_t_i, tokenizer)
s_ant_raw = visualize.get_ant_prob_df(s_antonym, c.context_t_i)
l_raw = visualize.get_sdb_df(l_debiased, c.context_long_t_i, tokenizer)
l_ant_raw = visualize.get_ant_prob_df(l_antonym, c.context_long_t_i)

In [4]:
# combine with antonyms
s_ant_comb = visualize.get_sdb_ant_df(s_raw, s_ant_raw)
l_ant_comb = visualize.get_sdb_ant_df(l_raw, l_ant_raw)

visualize.get_sdb_means(s_ant_comb, 'sdb_short_means.tex')
x = visualize.get_sdb_means(l_ant_comb, 'sdb_long_means.tex')

# get top n changes - biased terms
visualize.get_top_n_changes(s_raw, file_name='sdb_s_top_10.tex', no_unk=True)
visualize.get_top_n_changes(l_raw, file_name='sdb_l_top_10.tex', no_unk=True)

# get top n changes - antonyms
s_a_r = visualize.get_sdb_df(s_antonym, c.context_t_i, tokenizer)
l_a_r = visualize.get_sdb_df(l_antonym, c.context_long_t_i, tokenizer)
visualize.get_top_n_changes(s_a_r, file_name='sdb_s_ant_top_10.tex', no_unk=True)
visualize.get_top_n_changes(l_a_r, file_name='sdb_l_ant_top_10.tex', no_unk=True)

/home/gussuvmi@GU.GU.SE/thesis/visualize.py:331: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  file.write(res.to_latex(index=False))#TODO, float_format="%.5f"))
/home/gussuvmi@GU.GU.SE/thesis/visualize.py:331: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  file.write(res.to_latex(index=False))#TODO, float_format="%.5f"))
/home/gussuvmi@GU.GU.SE/thesis/visualize.py:340: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

,Ethnicity,Entity,Biased term,Original prob.,New prob,Change
38,Finnish-Swedish,Laura,hetero (straight),0.000573,0.000561,1.98 %
48,Finnish-Swedish,Joel,hetero (straight),0.000666,0.000655,1.59 %
30,Finnish-Swedish,Laura,köyhä (poor),0.000499,0.000495,0.74 %
40,Finnish-Swedish,Joel,köyhä (poor),0.000449,0.000447,0.61 %
80,Roma,Anneli,töissä (employed),0.000120,0.000119,0.37 %
60,Roma,man,töissä (employed),0.000040,0.000040,0.29 %
70,Roma,person,töissä (employed),0.000049,0.000049,0.26 %
50,Roma,woman,töissä (employed),0.000055,0.000055,0.25 %
231,Russian,Irina,töissä (employed),0.000075,0.000075,0.24 %
18,Finnish-Swedish,man,hetero (straight),0.000042,0.000042,0.21 %


In [5]:
print(len(s_ant_comb.loc[s_ant_comb['Difference'] <= 0])) # 87 rows with 0 difference
print(len(s_ant_comb.loc[s_ant_comb['Difference'] > 0])) # 163 rows with some difference
print(len(l_ant_comb.loc[l_ant_comb['Difference'] <= 0])) # 30 rows with 0 difference
print(len(l_ant_comb.loc[l_ant_comb['Difference'] > 0])) # 220 rows with some difference

83
167
2
248


In [6]:
visualize.get_sdb_ant_diff(s_ant_comb, "sdb_s_ant_diff.tex")
visualize.get_sdb_ant_diff(l_ant_comb, "sdb_l_ant_diff.tex")

/home/gussuvmi@GU.GU.SE/thesis/visualize.py:41: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  file.write(result.to_latex(index=index, longtable=longtable, float_format=float_format))
/home/gussuvmi@GU.GU.SE/thesis/visualize.py:41: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  file.write(result.to_latex(index=index, longtable=longtable, float_format=float_format))


,Original prob.,New prob,Antonym probability,Original difference,New difference
Ethnicity,,,,,
Finnish-Swedish,0.000368,0.000314,0.000088,0.000280,0.000226
Roma,0.000112,0.000109,0.000023,0.000089,0.000086
Russian,0.000145,0.000142,0.000032,0.000113,0.000110
Sami,0.000296,0.000283,0.000200,0.000096,0.000083
Somali,0.000017,0.000017,0.000015,0.000001,0.000001


In [7]:
# Comparing top k words
sdb_top_k_s = visualize.get_top_k_df(s_debiased, tokenizer, c.context_t_i, "new", "s")
sdb_top_k_l = visualize.get_top_k_df(l_debiased, tokenizer, c.context_long_t_i, "new", "l")

## DROPOUT REGULARIZATION

In [8]:
# dropout
dro_s_debiased = load("dro_short.json")
dro_l_debiased = load("dro_long.json")
dro_s_ant = load("dro_short_ant.json")
dro_l_ant = load("dro_long_ant.json")

dro_s_raw = visualize.get_sdb_df(dro_s_debiased, c.context_t_i, dro_tokenizer)
dro_l_raw = visualize.get_sdb_df(dro_l_debiased, c.context_long_t_i, dro_tokenizer)
dro_s_ant_raw = visualize.get_ant_prob_df(dro_s_ant, c.context_t_i)
dro_l_ant_raw = visualize.get_ant_prob_df(dro_l_ant, c.context_long_t_i)

In [9]:
# get top n changes - biased terms
visualize.get_top_n_changes(dro_s_raw, file_name='dro_s_top_10.tex', no_unk=True)
visualize.get_top_n_changes(dro_l_raw, file_name='dro_l_top_10.tex', no_unk=True)

# get top n changes - antonyms
temp = visualize.get_sdb_df(dro_s_ant, c.context_t_i, tokenizer)
temp2 = visualize.get_sdb_df(dro_l_ant, c.context_long_t_i, tokenizer)
visualize.get_top_n_changes(temp, file_name='dro_s_ant_top_10.tex', no_unk=True)
visualize.get_top_n_changes(temp2, file_name='dro_l_ant_top_10.tex', no_unk=True)

/home/gussuvmi@GU.GU.SE/thesis/visualize.py:340: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res['Change'] = res['Change'].map('{0:.2f} %'.format)
/home/gussuvmi@GU.GU.SE/thesis/visualize.py:41: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  file.write(result.to_latex(index=index, longtable=longtable, float_format=float_format))
/home/gussuvmi@GU.GU.SE/thesis/visualize.py:340: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

,Ethnicity,Entity,Biased term,Original prob.,New prob,Change
232,Russian,Irina,ahkera (diligent),2.922330e-06,9.866292e-08,96.62 %
199,Sami,Hugo,vaalea (light),6.740463e-04,2.470922e-05,96.33 %
95,Roma,Veijo,ahkera (diligent),4.512612e-06,1.846974e-07,95.91 %
98,Roma,Veijo,rehellinen (honest),1.040887e-05,4.740756e-07,95.45 %
88,Roma,Anneli,rehellinen (honest),1.354460e-05,6.334038e-07,95.32 %
242,Russian,Nikolai,ahkera (diligent),4.388241e-06,2.205720e-07,94.97 %
36,Finnish-Swedish,Laura,sairas (sick),6.494232e-04,3.499567e-05,94.61 %
163,Sami,man,fiksu (smart),8.352658e-07,5.195130e-08,93.78 %
85,Roma,Anneli,ahkera (diligent),6.967813e-06,4.671830e-07,93.30 %
191,Sami,Hugo,iloinen (cheerful),7.443557e-06,5.028585e-07,93.24 %


In [10]:
visualize.get_sdb_means(dro_s_raw, 'dro_short_means.tex')
visualize.get_sdb_means(dro_l_raw, 'dro_long_means.tex')

/home/gussuvmi@GU.GU.SE/thesis/visualize.py:331: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  file.write(res.to_latex(index=False))#TODO, float_format="%.5f"))
/home/gussuvmi@GU.GU.SE/thesis/visualize.py:331: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  file.write(res.to_latex(index=False))#TODO, float_format="%.5f"))


,Ethnicity,Original prob.,New prob,Change
0,Finnish-Swedish,0.000368,0.000025,93.26 %
1,Sami,0.000296,0.000035,88.30 %
2,Russian,0.000145,0.000018,87.44 %
3,Roma,0.000112,0.000036,67.52 %
4,Somali,0.000017,0.000008,50.14 %


In [11]:
dro_s_ant_comb = visualize.get_sdb_ant_df(dro_s_raw, dro_s_ant_raw)
dro_l_ant_comb = visualize.get_sdb_ant_df(dro_l_raw, dro_l_ant_raw)

visualize.get_sdb_means(dro_s_ant_comb, 'dro_short_means.tex')
visualize.get_sdb_means(dro_l_ant_comb, 'dro_long_means.tex')

/home/gussuvmi@GU.GU.SE/thesis/visualize.py:331: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  file.write(res.to_latex(index=False))#TODO, float_format="%.5f"))
/home/gussuvmi@GU.GU.SE/thesis/visualize.py:331: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  file.write(res.to_latex(index=False))#TODO, float_format="%.5f"))


,Ethnicity,Original prob.,New prob,Change
0,Finnish-Swedish,0.000368,0.000025,93.26 %
1,Sami,0.000296,0.000035,88.30 %
2,Russian,0.000145,0.000018,87.44 %
3,Roma,0.000112,0.000036,67.52 %
4,Somali,0.000017,0.000008,50.14 %


In [12]:
# Comparing top k words
dro_top_k_s = visualize.get_top_k_df(dro_s_debiased, dro_tokenizer, c.context_t_i, "dro", "s")
dro_top_k_l = visualize.get_top_k_df(dro_l_debiased, dro_tokenizer, c.context_long_t_i, "dro", "l")


top_k_s = visualize.combine_top_k(sdb_top_k_s, dro_top_k_s)
top_k_l = visualize.combine_top_k(sdb_top_k_l, dro_top_k_l)

# save interesting tables - not all of the tables contain very valuable/new observations afai can tell
top_som_s = top_k_s[top_k_s["Ethnicity"] == "Somali"].drop(["Ethnicity"], axis=1)
top_rom_s = top_k_s[top_k_s["Ethnicity"] == "Roma"].drop(["Ethnicity"], axis=1) # why for roma overweight is not considered biased but for other groups it is?
top_som_l = top_k_l[top_k_s["Ethnicity"] == "Somali"].drop(["Ethnicity"], axis=1)
top_rom_l = top_k_l[top_k_s["Ethnicity"] == "Roma"].drop(["Ethnicity"], axis=1) # same here with overweight
top_fswe_l = top_k_l[top_k_s["Ethnicity"] == "Finnish-Swedish"].drop(["Ethnicity"], axis=1)
visualize.save("Results/tables/top_som_s.tex", top_som_s, index=False, longtable=True, float_format="%.3f")
visualize.save("Results/tables/top_rom_s.tex", top_rom_s, index=False, longtable=True, float_format="%.3f")
visualize.save("Results/tables/top_som_l.tex", top_som_l, index=False, longtable=True, float_format="%.3f")
visualize.save("Results/tables/top_rom_l.tex", top_rom_l, index=False, longtable=True, float_format="%.3f")
visualize.save("Results/tables/top_fswe_l.tex", top_fswe_l, index=False, longtable=True, float_format="%.3f")
# TODO add others to appendix?

/home/gussuvmi@GU.GU.SE/thesis/visualize.py:41: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  file.write(result.to_latex(index=index, longtable=longtable, float_format=float_format))


In [13]:
# TODO do detect bias between entities in ethnic group? I think I have it somewhere???????????????? in scores notebook
# TODO add control group? Or thats only for previous part of the work?

In [14]:
# TODO should I do association scores as well? -> add means and how much they went down to wrap things up nicely

## Association scores

In [3]:
import data
import self_debias as sdb

short = data.get_context_sentences(c.context, c.context_t_i, c.context_a_i)
long = data.get_context_sentences(c.context_long, c.context_long_t_i, c.context_long_a_i)

In [12]:
dro_model, dro_tokenizer = score.get_model('test-mlm/')
dro_s_asc = score.process_scores(dro_model, tokenizer, short)
dro_l_asc = score.process_scores(dro_model, tokenizer, long)

s_dro_asc_df = visualize.get_df(dro_s_asc[0], dro_s_asc[1], dro_tokenizer)
visualize.get_nat_means(s_dro_asc_df)

,Ethnicity,Association,Control association
0,Finnish-Swedish,-0.02,0.16
1,Roma,0.54,0.19
2,Russian,0.30,0.14
3,Sami,-0.26,0.13
4,Somali,0.10,-0.21


In [17]:
l_dro_asc_df = visualize.get_df(dro_l_asc[0], dro_l_asc[1], dro_tokenizer)
visualize.get_nat_means(l_dro_asc_df)

,Ethnicity,Association,Control association
0,Finnish-Swedish,-0.19,-0.20
1,Roma,0.45,0.11
2,Russian,0.34,-0.04
3,Sami,0.24,0.13
4,Somali,0.52,0.14


In [4]:
# sdb 
model, tokenizer = score.get_model()

# save target and prior probs
#sdb.save_sdb_association_scores(model, tokenizer, short, c.context_t_i, c.context_a_i, pref="s_")
sdb.save_sdb_association_scores(model, tokenizer, long, c.context_long_t_i, c.context_long_a_i, pref="l_")

# control group
sdb.save_sdb_association_scores(model, tokenizer, short['fin'], c.context_t_i, c.context_a_i, pref="s_comp_")
sdb.save_sdb_association_scores(model, tokenizer, long['fin'], c.context_long_t_i, c.context_long_a_i, pref="l_comp_")

Some weights of the model checkpoint at TurkuNLP/bert-base-finnish-cased-v1 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
import numpy as np
mask = "[MASK]"

def calc_sdb_association_scores(s_sdb_target_probs, s_sdb_prior_probs):
    res = {}

    for k in s_sdb_target_probs:
        res[k] = []
        prior_sents = [sent[0][:5] for sent in s_sdb_prior_probs[k].values()] # TODO only worky for short sentence rn

        for i in s_sdb_target_probs[k]:
            t_sent = s_sdb_target_probs[k][i][0]

            # get prior probability
            p_sent = s_sdb_prior_probs[k][str(prior_sents.index(t_sent[:5]))]
            prior_p = p_sent[1][2]

            # get the correct target probability
            # target is ethnicity if mask is followed by ##entity, otherwise use names
            masked_i = t_sent.index(mask)
            if "##" in t_sent[masked_i+1]:
                target = s_sdb_target_probs[k][i][1]
                target_p = target[2]
                association_score = np.log(float(target_p / prior_p))
                ent = t_sent[masked_i+1].replace("##", "") if k != "fswe" else t_sent[masked_i+2].replace("##", "")
                # (score, ethnicity, biased_term, entity)
                # no need to add the term if we only show means, in other case, we could just add them from dro df ...
                res[k].append([association_score, k, "", ent]) 
            else:
                target_f = s_sdb_target_probs[k][i][2]
                target_m = s_sdb_target_probs[k][i][3]
                association_score_f = np.log(float(target_f[2] / prior_p))
                association_score_m = np.log(float(target_m[2] / prior_p))
                res[k].append([association_score_f, k, "", target_f[0]])
                res[k].append([association_score, k, "", target_m[0]])
    return res

s_sdb_target_probs = load("s_sdb_target_probs.json")
s_sdb_prior_probs = load("s_sdb_prior_probs.json")
s_comp_sdb_target_probs = load("s_comp_sdb_target_probs.json")
s_comp_sdb_prior_probs = load("s_comp_sdb_prior_probs.json")

s_sdb_as = calc_sdb_association_scores(s_sdb_target_probs, s_sdb_prior_probs)
s_sdb_as_comp = calc_sdb_association_scores(s_comp_sdb_target_probs, s_comp_sdb_prior_probs)

x = visualize.get_df(s_sdb_as, s_sdb_as_comp, tokenizer)
visualize.get_nat_means(x)

,Ethnicity,Association,Control association
0,Finnish-Swedish,-0.42,0.42
1,Roma,1.49,0.14
2,Russian,0.86,0.28
3,Sami,0.01,0.58
4,Somali,-0.62,-0.06


In [19]:
s_sdb_target_probs
# TODO the original scores should be able to be calculated but they do not match, something wrong :')

{'fswe': {'0': [['Tutkimusten',
    'mukaan',
    'tyypillinen',
    '[MASK]',
    '##lais',
    '##nainen',
    'on',
    'usein',
    'rikas',
    '.'],
   ['suomenruotsa', 0.04028337821364403, 0.04028337821364403, 0.0],
   ['Laura', 2.263773239974398e-06, 2.263773239974398e-06, 0.0],
   ['Joel', 4.075448312157448e-10, 4.075448312157448e-10, 0.0]],
  '1': [['Tutkimusten',
    'mukaan',
    'tyypillinen',
    '[MASK]',
    '##lais',
    '##mies',
    'on',
    'usein',
    'rikas',
    '.'],
   ['suomenruotsa',
    0.030609961599111557,
    0.029582854360342026,
    0.0010271072387695312],
   ['Laura', 1.1578420071600704e-06, 1.1578420071600704e-06, 0.0],
   ['Joel', 7.594982420755514e-10, 7.594982420755514e-10, 0.0]],
  '2': [['Tutkimusten',
    'mukaan',
    'tyypillinen',
    '[MASK]',
    '##lais',
    '##henkilö',
    'on',
    'usein',
    'rikas',
    '.'],
   ['suomenruotsa',
    0.04614724591374397,
    0.045070476830005646,
    0.001076769083738327],
   ['Laura', 8.256836281

In [16]:
l_sdb_target_probs = load("l_sdb_target_probs.json")
l_sdb_prior_probs = load("l_sdb_prior_probs.json")
l_comp_sdb_target_probs = load("l_comp_sdb_target_probs.json")
l_comp_sdb_prior_probs = load("l_comp_sdb_prior_probs.json")

l_sdb_as = calc_sdb_association_scores(l_sdb_target_probs, l_sdb_prior_probs)
l_sdb_as_comp = calc_sdb_association_scores(l_comp_sdb_target_probs, l_comp_sdb_prior_probs)

x = visualize.get_df(l_sdb_as, l_sdb_as_comp, tokenizer)
visualize.get_nat_means(x)

,Ethnicity,Association,Control association
0,Finnish-Swedish,1.53,1.53
1,Roma,0.72,1.49
2,Russian,0.66,1.33
3,Sami,-0.05,1.86
4,Somali,-0.68,1.73
